In [1]:
import torch_geometric
from torch_geometric.datasets import Planetoid

In [2]:
#root: create new folder for downloaded data
#이미 있으면 아무것도 안나옴
dataset=Planetoid(root="tutorial", name="Cora")


Processing...
Done!


In [26]:
print(dataset)
print("# of graphs:\t",len(dataset)) 
print("# of classes:\t", dataset.num_classes)
print("# of node features:\t",dataset.num_node_features)
print("# of edge features:\t",dataset.num_edge_features)

Cora()
# of graphs:	 1
# of classes:	 7
# of node features:	 1433
# of edge features:	 0


In [27]:
#Summarization of dataset
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [28]:
print("edge index:\t", dataset.data.edge_index.shape)
#edge info : first row and second row are the ends of edges
print(dataset.data.edge_index)
# number of edges in the graph
print(len(dataset.data.edge_index[0]))

edge index:	 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])
10556


In [29]:
# Train mask: tensor filled with boolean values
# indicate whether a node is in the training set
print("train mask:\t",dataset.data.train_mask.shape)
print(dataset.data.train_mask)

train mask:	 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


In [30]:
#X : node feature matrix
print("X:\t",dataset.data.x.shape)
print(dataset.data.x)
# y: node label
print("y:\t", dataset.data.y.shape)
print(dataset.data.y)

X:	 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
y:	 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [31]:
import os.path as osp
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [32]:
#I only have one dataset : Cora
data=dataset[0]

In [33]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv=SAGEConv(dataset.num_features,
                           dataset.num_classes,
                           aggr="max")
    def forward(self):
        x=self.conv(data.x, data.edge_index)
        return F.log_softmax(x, dim=1)

In [34]:
# GPU : check if GPU is available. select gpu to use
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data=Net().to(device),data.to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=0.01,weight_decay=5e-4) 

In [37]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask],data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs=model(),[]
    for _, mask in data('train_mask','val_mask','test_mask'):
        pred=logits[mask].max(1)[1]
        acc=pred.eq(data.y[mask]).sum().item()/mask.sum().item()
        accs.append(acc)
    return accs

In [38]:
best_val_acc=test_acc=0
for epoch in range(1,100):
    train()
    _, val_acc, tmp_test_acc=test()
    if val_acc>best_val_acc:
        best_val_acc=val_acc
        test_acc=tmp_test_acc
    log='Epoch{:03d}, Val: {:.4f}, Test:{:.4f}'
    
    if epoch%10==0:
        print(log.format(epoch,best_val_acc,test_acc))

Epoch010, Val: 0.7280, Test:0.7250
Epoch020, Val: 0.7280, Test:0.7250
Epoch030, Val: 0.7280, Test:0.7250
Epoch040, Val: 0.7280, Test:0.7250
Epoch050, Val: 0.7280, Test:0.7250
Epoch060, Val: 0.7320, Test:0.7130
Epoch070, Val: 0.7320, Test:0.7130
Epoch080, Val: 0.7320, Test:0.7130
Epoch090, Val: 0.7320, Test:0.7130
